In [37]:
#!pip install ipykernel
import duckdb
import ibis
import polars as pl
from openai import OpenAI
import json
import pandas as pd
from summarytools import dfSummary
import matplotlib.pyplot as plt

In [25]:
# Connect to your DuckDB database using Ibis
con = ibis.duckdb.connect('samhsa_data.db')
con.list_tables() # List the tables in the db")


['Demographics',
 'EmploymentDetails',
 'LegalInfo',
 'SubstanceUseHistory',
 'TreatmentInformation',
 'ValueLabelMapping',
 'teds_a_raw_2015_2019']

#### Data Ingestion : Creating Polars DataFrame 

We could use two apporaches here. USe a simple ibis connection to excute the select query and turn the resulting pandas frame into polars. 
OR use duckdb connection and arrow to input directly into polars without a panda - polar connversion. Approach 2  appears to be marginally faster for this amount of data. So I would be going ahead with that for the future tables. 


In [26]:
# Ibis method
demographics_pd = con.table('Demographics').execute() #you can add limit(100) before excecute to sample this data
# And then convert to polars 
demo_pl = pl.from_pandas(demographics_pd)


In [27]:
demo_pl.head(2)
#demo_pl.columns

CASEID,ADMYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,PREG,VET,STFIPS,CBSA2010,REGION,DIVISION
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
20151524993,2015,6,2,5,3,4,2,1,2,2,-9,4,9
20151449532,2015,7,2,1,4,2,3,2,2,2,-9,4,9


In [28]:
# DuckDB Method to Load database table 
dbcon = duckdb.connect('samhsa_data.db')
demo = pl.from_arrow(dbcon.execute("SELECT * FROM Demographics").arrow())
dbcon.close()

In [29]:
demo.head(2)
#pl_df.columns

CASEID,ADMYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,PREG,VET,STFIPS,CBSA2010,REGION,DIVISION
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
20151524993,2015,6,2,5,3,4,2,1,2,2,-9,4,9
20151449532,2015,7,2,1,4,2,3,2,2,2,-9,4,9


In [30]:
# Ingest all the other tables into polars dfs
dbcon = duckdb.connect('samhsa_data.db')
emp_det = pl.from_arrow(dbcon.execute("SELECT * FROM EmploymentDetails").arrow())
legal_info = pl.from_arrow(dbcon.execute("SELECT * FROM LegalInfo").arrow())
subs_hist = pl.from_arrow(dbcon.execute("SELECT * FROM SubstanceUseHistory").arrow())
treat_info = pl.from_arrow(dbcon.execute("SELECT * FROM TreatmentInformation").arrow())
value_label = pl.from_arrow(dbcon.execute("SELECT * FROM ValueLabelMapping").arrow())
dbcon.close()


In [31]:
# Checking if update on value_label worked
value_label.tail(10)


id,value,label,tablename,date_added
i32,i32,str,str,date
45,3,"""Retired, disabled""","""DETNLF""",2024-07-16
46,4,"""Resident of institution""","""DETNLF""",2024-07-16
47,5,"""Other""","""DETNLF""",2024-07-16
48,-9,"""Missing/unknown/not collected/…","""DETNLF""",2024-07-16
49,1,"""Yes""","""PREG""",2024-07-16
50,2,"""No""","""PREG""",2024-07-16
51,-9,"""Missing/unknown/not collected/…","""PREG""",2024-07-16
52,1,"""Yes""","""VET""",2024-07-16
53,2,"""No""","""VET""",2024-07-16


#### Viewing high level summary of tables 

Visualize genral stats for columns across tables using SummaryTools and Describe Functions

##### Building custom function to do a pandas profiling type analysis with Polars

In [45]:

def eda_polars(df: pl.DataFrame):
    # Display the first few rows
    print("First few rows of the DataFrame:")
    print(df.head())
    print("\n")
    
    # Summary statistics
    print("Summary Statistics:")
    summary = df.describe()
    print(summary)
    print("\n")
    
    # Column names and types
    print("Column Names and Types:")
    print(df.schema)
    print("\n")
    
    # Distribution of categorical features
    print("Distribution of Categorical Features:")
    for col in df.columns:
        if df[col].dtype == pl.Utf8:
            print(f"\nValue counts for {col}:")
            print(df.select(pl.col(col).value_counts()))
    print("\n")
    
    # Missing values
    print("Missing Values:")
    missing_values = df.null_count()
    print(missing_values)
    print("\n")
    
    # Correlation matrix for numerical features
    print("Correlation Matrix:")
    numeric_cols = [col for col in df.columns if df[col].dtype in [pl.Int64, pl.Float64]]
    if numeric_cols:
        corr_matrix = df.select(numeric_cols).corr()
        print(corr_matrix)
        print("\n")
    
 



In [47]:
# Example data
eda_polars(subs_hist)


First few rows of the DataFrame:
shape: (5, 33)
┌─────────────┬───────┬──────┬────────┬───┬──────────┬────────┬────────┬──────────┐
│ CASEID      ┆ ADMYR ┆ SUB1 ┆ ROUTE1 ┆ … ┆ SEDHPFLG ┆ INHFLG ┆ OTCFLG ┆ OTHERFLG │
│ ---         ┆ ---   ┆ ---  ┆ ---    ┆   ┆ ---      ┆ ---    ┆ ---    ┆ ---      │
│ i64         ┆ i64   ┆ i64  ┆ i64    ┆   ┆ i64      ┆ i64    ┆ i64    ┆ i64      │
╞═════════════╪═══════╪══════╪════════╪═══╪══════════╪════════╪════════╪══════════╡
│ 20151524993 ┆ 2015  ┆ 2    ┆ 1      ┆ … ┆ 0        ┆ 0      ┆ 0      ┆ 0        │
│ 20151449532 ┆ 2015  ┆ 5    ┆ 3      ┆ … ┆ 0        ┆ 0      ┆ 0      ┆ 0        │
│ 20151431263 ┆ 2015  ┆ 3    ┆ 3      ┆ … ┆ 0        ┆ 0      ┆ 0      ┆ 0        │
│ 20151401765 ┆ 2015  ┆ 2    ┆ 1      ┆ … ┆ 0        ┆ 0      ┆ 0      ┆ 0        │
│ 20151461771 ┆ 2015  ┆ 5    ┆ 4      ┆ … ┆ 0        ┆ 0      ┆ 0      ┆ 0        │
└─────────────┴───────┴──────┴────────┴───┴──────────┴────────┴────────┴──────────┘


Summary Statistics:
shape:

In [12]:
# Store your credentials in a json locally | Don't forget to add this file to your gitignore

credentials_path = 'credentials.json'

# Read Credential
with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Pass this key to configure OpenAI's API client
client = OpenAI(api_key=openai_api_key)



In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)